# Full Runnable System

## Wifi Sniffing on a Mac Book

It is possibleto do the sniffing piece through Python code using the **scapy** package, but I have found that the easiest way is to do it is to use the Mac book's internal network hardware. You need a network card that supports *monitor mode* and not all routers do - luckily the internal Mac Book network card does. 

To put the card into monitor mode and start capturing wifi packets you need to right click on the wifi icon with the **alt** key held done. This shows an advanced menu from which you can "*Open Wireless Diagnostics*". 

![Screenshot 1](./wifiSniffingScreen1.png)

Once the Wireless Diagnostics application has stared the **Window** menu allows you to open up the *Sniffer* application - this does the sniffing!

![Screenshot 2](./wifiSniffingScreen2.png)

I don't know as much about Wifi channels as I should. The sniffer application just uses whichever channel you are connected to. If doing the sniffing through code you can set up channel hopping, or high end sniffers (yikes!) will use multiple network cards to sniff on multiple channel simultaneously. If have found, however, that using whatever channel pops up here gives grand results for demonstrations.


![Screenshot 4](./wifiSniffingScreen4.png)

Click start and sniffing will begin. You'll notice that the wifi icon changes to show that your hardware is in monitor mode. 

![Screenshot 4](./wifiSniffingScreen5.png)

I usually leave this run for about 10 minutes and then press stop. It dumps out all of the packets sniffed into a **.wcap** file on your desktop named using a date time stamp, e.g. 2016.03.13_16-53-28-GMT+1.wcap. Watch out, if you leave the sniffer run for too long this file gets very big.

You can open this file and look at its contents using Wireshark (https://www.wireshark.org/). To find mobile phones, laptops, tablets etc (any wireless access point) you need to find **Probe Requests** which are a specific type of packet that is part of the 802.11 management protocol. In the the 802.11 a management packet has a type of 0 and a probe request has a subtype of 4. 

The Python code below opens up the wcap file and processes it to find the probe requests and display them. There are a couple of other neat things you can do. Probe requests include the MAC address for the device. Some manufacturers have blocks of MAC addresses associated with them, so you can match MAC addresses to devices by looking this up. Also, as part of a the probe rewuest protocol some OSs broadcast the names of the networks to which they frequently/recently connect (I am not sure about the exact rules governing this) which we can extract. Due to people doing **wardriving** (https://en.wikipedia.org/wiki/Wardriving) there are large databases of the locations of wifi networks so you can geolocate these networks and display them on a map - which is a neat trick. Wigle.net (https://wigle.net/) is a service that allows look up of a database of wifi network locations collected by people and is used in the code below.

## Setup

Import various packages used in the code below

In [4]:
from scapy.all import * # for .wcap file handling
import glob, os # for file handling
import csv # for handling csv files
import time # for messing around with time formats
from datetime import datetime  # for generating datetimes and formatting them
import json # for parsing json files
import webbrowser # allows HTML pages to be opened in the default webbrowser

Load up the details of manufacturers into a Python dictioanry so we can look this up easily.

In [5]:
MAC_manufacturers = dict()
for line in open("manufOriginal.txt"):
    words1 = line.split("\t")
    if(len(words1) > 1):
        words2 = words1[1].split("#")
        MAC_manufacturers[words1[0]] = words2[0].strip()

## Open & Process The .wcap File

Open the latest .wcap file on the dekstop.

In [6]:
oldFolder = os.getcwd()
# Fix this path!!!
os.chdir("/Users/bmacnamee/Desktop/")
wcapFiles = glob.glob("*.wcap")
wcapFiles.sort()
wcapFiles.reverse()
pkts = rdpcap(wcapFiles[0])
print(len(pkts))
os.chdir(oldFolder)

10674


Process packets list to find probe requests

In [7]:
devices = dict()

counter = 0

# Loop through the packets in the file and take a look at them
for pkt in pkts:

    if pkt.haslayer(Dot11) :
        # Type = 0 is a management frame
        # Subtype 8 is a beacon frame
        # Subtype 4 is a probe request frame
        # Subtype 5 is a probe response frame
        if pkt.type == 0 and pkt.subtype == 4 :

            counter += 1
           
            if not pkt.addr2 in devices:
                
                # Isolate the manufacturers code
                manuCode = pkt.addr2[0:8]
                if manuCode in MAC_manufacturers:
                    manu = MAC_manufacturers[manuCode]
                else:
                    manu = ""
                    
                devices[pkt.addr2] = {"freq":1, "last_seen":pkt.time, "manu":manu, "ssids":set()}

                # If there is an SSID listed (in info attribute) record it
                if hasattr(pkt, 'info'):
                    ssid = pkt.info
                    ssid = ssid.strip()
                    ssid = ssid.replace("&", " ")
                    #ssid = ''.join(filter(lambda x:x in string.printable, ssid))
                    
                    # Not very good, but simple error handling - ssids of more than 32 charcters probably indicate malformed packets.
                    #if(len(ssid) <= 32 and ssid.find("\x00") == -1):
                    if(all(c in string.printable for c in ssid) and (len(ssid) > 5)):
                        devices[pkt.addr2]["ssids"].add(ssid)

            else:
                # Increment appearance frequency
                devices[pkt.addr2]["freq"] += 1

                # If this occurrence is more recent that thae recorded one update last_seen
                if devices[pkt.addr2]["last_seen"] < pkt.time:
                    devices[pkt.addr2]["last_seen"] = pkt.time

                # If there is an SSID listed (in info attribute) record it
                if hasattr(pkt, 'info'):
                    ssid = pkt.info
                    ssid = ssid.strip()
                    #ssid = ''.join(filter(lambda x:x in string.printable, ssid))
                    # Not very good, but simple error handling - ssids of more than 32 charcters probably indicate malformed packets.
                    #if(len(ssid) <= 32 and ssid.find("\x00") == -1):
                    if(all(c in string.printable for c in ssid) and (len(ssid) > 5)):
                        devices[pkt.addr2]["ssids"].add(ssid)
                        

print(str(len(devices)) + " found")


33 found


Write out the devices read in into a nice csv format

In [8]:
with open('devices' +  str(time.time()) + '.csv', 'wb') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['ID', 'freq', 'manu', 'ssids', 'last_seen'])
    
    for d in devices:
        ssidString = " | ".join(str(e) for e in devices[d]['ssids'])
        writer.writerow([d, devices[d]['freq'] , devices[d]['manu'], ssidString, devices[d]['last_seen']])                      

Generate a list of the ssids found

In [9]:
ssidsFound = []

# Iterate through the devices list
for d in devices:
    
    # get the list of ssids saved from a device
    ssids = devices[d]["ssids"]
    
    # loop through the SSIDs
    for ssid in ssids:

        ssidsFound.append(ssid)

ssidsFound = list(set(ssidsFound))
print(len(ssidsFound))
print(ssidsFound)

9
['Kleinova_AP', 'Cafe_Viena', 'wifi-guest', 'wifi-cheval-blanc', 'SFR WiFi FON', 'orange-4361', 'Kelner', 'ALBATROS_300', 'WIFI-ANTARES']


## Generate HTML Page

Apologies for awful approach to generating a HTML file - I'm sure there is a much better way!!

In [10]:
devicesPerRow = 3
numDevicesFound = len(devices)

deviceCount = 0

with open('devicesFound.html', 'w') as htmlfile:
    
    # Write the HTML header
    htmlfile.write("<!DOCTYPE html>\n")
    htmlfile.write("<html lang=\"en\">\n")
    htmlfile.write("<head>\n")
    htmlfile.write("<title>Devices Found</title>\n")
    htmlfile.write("<meta charset=\"utf-8\">\n")
    htmlfile.write("<meta name=\"viewport\" content=\"width=device-width, initial-scale=1\">\n")
    htmlfile.write("<link rel=\"stylesheet\" href=\"http://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/css/bootstrap.min.css\">\n")
    htmlfile.write("<script src=\"https://ajax.googleapis.com/ajax/libs/jquery/1.12.0/jquery.min.js\"></script>\n")
    htmlfile.write("<script src=\"http://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/js/bootstrap.min.js\"></script>\n")
    htmlfile.write("<style type=\"text/css\"> h1.panel-title {color:#ffffff; font-size:32px}  </style>")
    htmlfile.write("<style type=\"text/css\"> p {font-size:24px}  </style>")
        
    htmlfile.write("</head>\n")
    htmlfile.write("<body>\n")
    
    htmlfile.write("<p>&nbsp;</p>\n")
    
    htmlfile.write("<h2>&nbsp;&nbsp; Found " + str(len(devices)) + " devices nearby.</h2>\n")
    
    htmlfile.write("<p>&nbsp;</p>\n")
    
    # Start the devices block
    htmlfile.write("<div class=\"container-fluid\">\n")
    
    # Loop through each device
    for d in devices:
    #for d in sorted(devices, key=devices.get, reverse=True):
        
        # If we need to start a new row start one
        if (deviceCount % devicesPerRow == 0):
            htmlfile.write("\t<div class=\"row\">\n")
        
        # Write our the details of a device
        #htmlfile.write("\t\t<div class=\"col-sm-3\" style=\"border:solid 2px black;background-color:grey;\" >\n")
        htmlfile.write("\t\t<div class=\"col-sm-4\">\n")
        htmlfile.write("<div class = \"panel panel-default\">\n")
        htmlfile.write("<div class = \"panel-heading\" style = \"background-color:#0033cc;\">\n")

        htmlfile.write("<h1 class = \"panel-title\">" + d + "</h1>\n")
        htmlfile.write("</div>\n")
   
        htmlfile.write("<div class = \"panel-body\">\n")

        if(devices[d]['freq'] <= 5):
            htmlfile.write("\t\t\t<img src = \"./WiFi-Signal-Strength-Low.jpg\" width = \"60\"/>\n")
        elif(devices[d]['freq'] <= 10):
            htmlfile.write("\t\t\t<img src = \"./WiFi-Signal-Strength-Lowish.jpg\" width = \"60\" />\n")
        elif(devices[d]['freq'] <= 50):
            htmlfile.write("\t\t\t<img src = \"./WiFi-Signal-Strength-Med.jpg\" width = \"60\" />\n")
        else:
            htmlfile.write("\t\t\t<img src = \"./WiFi-Signal-Strength-High.jpg\" width = \"60\" />\n")
            
        if devices[d]['manu'].find("Apple") >= 0:
            htmlfile.write("\t\t\t<img src = \"./Apple.png\" width = \"40\"/>\n")
        elif devices[d]['manu'].startswith("Htc"):
            htmlfile.write("\t\t\t<img src = \"./HTC.jpg\" height = \"30\" />\n")
        elif devices[d]['manu'].find("Samsung") >= 0:
            htmlfile.write("\t\t\t<img src = \"./samsung.png\" height = \"50\"/>\n")
        elif devices[d]['manu'].find("Sony") >= 0:
            htmlfile.write("\t\t\t<img src = \"./sony.jpg\" height = \"40\"/>\n")

        htmlfile.write("\t\t\t<span style=\"font-size:20px;\" >" + datetime.fromtimestamp(devices[d]['last_seen']).strftime('%Y-%m-$d %H:%M:%S').split(' ')[1] + "</span>\n")
  
        #htmlfile.write("\t\t\t<h3>Strength: " + str(devices[d]['freq']) + "</h3>\n")

        #htmlfile.write("\t\t\t<h3>" + datetime.fromtimestamp(devices[d]['last_seen']).strftime('%Y-%m-$d %H:%M:%S').split(' ')[1] + "</h3>\n")
        htmlfile.write("\t\t\t<br />\n")
        htmlfile.write("\t\t\t<br />\n")
        
        ssidString = ", ".join(str(e) for e in devices[d]['ssids'])
        htmlfile.write("\t\t\t<p>" + ssidString + "</p>\n")
        htmlfile.write("</div>\n")
        htmlfile.write("</div>\n")

        #htmlfile.write("\t\t\t<h1>" + d + "</h1>\n")
        #htmlfile.write("\t\t\t<h2>Handset: " + devices[d]['manu'] + " </h2>\n")
        #htmlfile.write("\t\t\t<h2>Strength: " + str(devices[d]['freq']) + "</h2>\n")
        #htmlfile.write("\t\t\t<h2>" + str(devices[d]['last_seen']) + "</h2>\n")
        #ssidString = ", ".join(str(e) for e in devices[d]['ssids'])
        #htmlfile.write("\t\t\t<p>" + ssidString + "</p>\n")
        htmlfile.write("\t\t</div>\n")
        

        deviceCount += 1
        
        # If we need to end a row end one
        if (deviceCount % devicesPerRow == 0):
            htmlfile.write("\t</div>\n")
    
    # If we didn't just end a row, end one
    if (deviceCount % devicesPerRow != 0):
        htmlfile.write("\t</div>\n")
    
    # End the block with a </div>
    htmlfile.write("</div>\n")
    

    htmlfile.write("</body>\n")
    htmlfile.write("</html>\n")

## Generate Wigle Map

I have downloaded a big collection of lat/long pairs for Irish wifi networks from Wigle to a json file. Due to rate limiting of the Wigle API, however, I have gotten them all. so we read the lats/longs avialble from the wigle dump and average out lats/longs where ssid show up multiple times.

In [11]:
# Import the Wigle dump
ssidList = json.loads(open('wigleFullDump2.json').read())

# Start an empty dictionary
ssidLatLong = dict()

# Loop through each ssid json object and add it to the dictionary
for ssidObj in ssidList:

    # Extract the important details from the ssid json object
    ssidName = ssidObj["ssid"] 
    ssidLat = ssidObj["trilat"]
    ssidLong = ssidObj["trilong"]
    
    # if the ssid is not in the dictionary yet add it
    if not ssidName in ssidLatLong:
        
        ssidLatLong[ssidName] = {"lat":ssidLat, "long":ssidLong, "count":1}

    # if the ssid is already in the ditionary then average out this new entry
    else:
        
        ssidLatLong[ssidName]["lat"] = (ssidLatLong[ssidName]["lat"]*ssidLatLong[ssidName]["count"] + ssidLat)/(ssidLatLong[ssidName]["count"] + 1)
        ssidLatLong[ssidName]["long"] = (ssidLatLong[ssidName]["long"]*ssidLatLong[ssidName]["count"] + ssidLong)/(ssidLatLong[ssidName]["count"] + 1)
        ssidLatLong[ssidName]["count"] += 1
        
# Manually fix UCD Wireless
ssidLatLong["UCD Wireless"]["lat"] = 53.306688
ssidLatLong["UCD Wireless"]["long"] = -6.220993
ssidLatLong["UCD Wireless"]["count"] = 100

print(len(ssidLatLong))



6968


Write out an XML file containing lat/longs of ssids spotted. Apologies for awful approach to generating an XML file - I'm sure there is a much better way!!

In [12]:
# Horrible manual writing of an XML file!!!!
with open('ssidsFound.xml', 'w') as xmlfile:
        
    xmlfile.write("<?xml version=\"1.0\"?>\n")
    xmlfile.write("<markers>\n")
    
    for ssid in ssidsFound:    
        if ssid in ssidLatLong and ssid != None:
            xmlfile.write("\t<marker>\n")
            xmlfile.write("\t\t<name>" + ssid + "</name>\n")
            xmlfile.write("\t\t<lat>" + str(ssidLatLong[ssid]["lat"]) + "</lat>\n")
            xmlfile.write("\t\t<lng>" + str(ssidLatLong[ssid]["long"]) + "</lng>\n")
            if ssid.startswith("UPC") or ssid.startswith("eircom") or ssid.startswith("vodafone") or ssid.startswith("BTHub"):
                xmlfile.write("\t\t<colour>green</colour>\n")
            else:
                xmlfile.write("\t\t<colour>red</colour>\n")
            xmlfile.write("\t</marker>\n")
        else:
            print(ssid)
    xmlfile.write("</markers>\n")


Kleinova_AP
Cafe_Viena
wifi-guest
wifi-cheval-blanc
SFR WiFi FON
orange-4361
Kelner
ALBATROS_300
WIFI-ANTARES


Print out the details of the devices found.

In [13]:
for d in devices:
    ssidString = " | ".join(str(e) for e in devices[d]['ssids'])
    print(d + " " + str(devices[d]['freq']) + " " + devices[d]['manu'] + " " + ssidString + " " + str(devices[d]['last_seen']))

c8:b5:b7:0d:b8:35 6  wifi-guest 1457884462.29
78:f7:be:53:91:ef 11  SFR WiFi FON 1457884481.39
4a:48:60:73:70:a2 2   1457884422.36
e4:32:cb:40:af:a7 6   1457884469.46
fe:d0:6f:aa:58:58 1   1457884453.3
3a:ef:e4:c4:86:1c 4   1457884439.91
e8:50:8b:fb:b4:3e 13  wifi-guest 1457884477.82
04:c2:3e:2c:ff:9a 1   1457884412.89
d4:8f:33:a1:c5:65 4   1457884474.53
10:30:47:43:f9:66 9 SamsungE  1457884475.14
9c:f3:87:29:70:43 2  wifi-guest 1457884424.07
80:6a:b0:10:43:59 1   1457884432.02
48:86:e8:26:2a:50 2   1457884430.55
e0:66:78:66:63:8c 1   1457884411.94
b2:ea:22:4a:18:c3 1   1457884442.41
d8:9e:3f:c7:46:f2 11  Kleinova_AP | Cafe_Viena | orange-4361 | Kelner | ALBATROS_300 | WIFI-ANTARES 1457884461.27
ac:61:ea:f2:d8:5d 6  wifi-guest 1457884486.22
00:ee:bd:ae:54:63 26  wifi-guest 1457884481.06
38:ca:da:3b:4a:52 27  wifi-guest 1457884486.37
ac:cf:5c:87:f9:20 6  wifi-guest 1457884484.68
42:11:1a:47:fe:ea 1   1457884416.95
42:0c:c5:b7:32:26 1   1457884435.41
48:74:6e:97:47:2f 1  wifi-cheval-blan

## Open Display Pages

The code above should have generated two HTML files, open them. The hardocded paths here is a bit flakey!

In [14]:
# Fix path!!!
url2 = 'file:///Users/bmacnamee/Dropbox/Research/Projects/WifiSniffing/markers.html'
webbrowser.open(url2)

# Fix path!!!
url1 = 'file:///Users/bmacnamee/Dropbox/Research/Projects/WifiSniffing/devicesFound.html'
webbrowser.open(url1)

True